Cellario U-Net

In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

from IPython.display import clear_output
from skimage.io import imread
from skimage.transform import resize


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#!ls drive/'My Drive'
!ls drive/'My Drive/Gland_dataset' &> /dev/null

drive_path = 'drive/My Drive/Cellari_project'

In [4]:
#seperate the data into training and test and annotated data and load it
#60 test A images
# 20 test B images
# 85 train images

test_images_A = [imread(drive_path +'/Gland_dataset/testA_{}.bmp'.format(i)) for i in range(1,61)] 
test_images_B = [imread(drive_path +'/Gland_dataset/testB_{}.bmp'.format(i)) for i in range(1,21)] 
test_images = test_images_A + test_images_B
print("done with test images")

test_images_A_anno = [imread(drive_path +'/Gland_dataset/testA_{}_anno.bmp'.format(i)) for i in range(1,61)] 
test_images_B_anno = [imread(drive_path +'/Gland_dataset/testB_{}_anno.bmp'.format(i)) for i in range(1,21)]   
test_images_anno = test_images_A_anno + test_images_B_anno 
print("done with test images annotated")

train_images = [imread(drive_path +'/Gland_dataset/train_{}.bmp'.format(i)) for i in range(1,86)] 
print("done with train images")
train_images_anno = [imread(drive_path +'/Gland_dataset/train_{}_anno.bmp'.format(i)) for i in range(1,86)]
print("done with train images annotated")

print("The training set contains {} images".format(len(train_images)))
print("The test set contains {} images".format(len(test_images )))

done with test images
done with test images annotated
done with train images
done with train images annotated
The training set contains 85 images
The test set contains 80 images


In [7]:
#reshaped image for to make the convolutions easier
test_images = np.array(test_images)
for i in range(0,len(test_images)):
  test_images[i] = resize(test_images[i], output_shape=(522,522), mode='reflect', anti_aliasing=True)
  #test_images_mean = np.mean(test_images[i],axis=(0,1))
  #test_images[i] = (test_images[i] - test_images[i].mean() )/test_images[i].std()
  
  
  test_images_anno[i] = resize(test_images_anno[i], output_shape=(522,522), mode='reflect', anti_aliasing=True)
  
for i in range(0,len(train_images)):
  train_images[i] = resize(train_images[i], output_shape=(522,522), mode='reflect', anti_aliasing=True)
  train_images_anno[i] = resize(train_images_anno[i], output_shape=(522,522), mode='reflect', anti_aliasing=True)


In [34]:
print(test_images[0][:][:][0])

test =test_images[0][:][:][0]
print(test.shape)

[[ 0.86196347  0.43647086  0.60580816]
 [ 0.49659997 -0.19417281  0.52658541]
 [-0.35185351 -1.38061522  0.18875472]
 ...
 [ 0.77438067  0.18775257  0.6769082 ]
 [-0.10927999 -0.95781259  0.46147191]
 [ 0.35809203 -0.09155773  0.2111449 ]]
(522, 3)


In [ ]:
#reshaped image for to make the convolutions easier
test_images = np.array(test_images)
for i in range(0,len(test_images)):
  test_images[i] = resize(test_images[i], output_shape=(522,522), mode='reflect', anti_aliasing=True)
  test_images_mean = np.mean(test_images[i],axis=(0,1))
  test_images[i] = (test_images[i] - test_images[i].mean() )/test_images[i].std()

In [ ]:
# Plot of the first image
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.imshow(train_images[0])
plt.title("Train image 1")
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(train_images_anno[0])
plt.title("Annotation for train image 1 ")
plt.show()

# Build the model

The U-Net architecture


In [ ]:
# Load functions
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import Linear, Conv2d, Dropout, MaxPool2d, BatchNorm1d
from torch.nn.functional import relu, sigmoid, tanh, softmax

In [ ]:
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")


def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x


def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

In [ ]:

def double_conv(channel_in, channel_out):
  conv = nn.Sequential(
      nn.Conv2d(channel_in, channel_out, kernel_size = 3),
      nn.ReLU(inplace=True),
      nn.Conv2d(channel_out, channel_out, kernel_size = 3),
      nn.ReLU(inplace=True)
  )
  return conv




class U_Net(nn.Module):
    def __init__(self):
        super(U_Net, self).__init__()
 
        self.max_pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

        #convolute the image down 
        self.conv_down_1 = double_conv(3, 64)
        self.conv_down_2 = double_conv(64, 128)
        self.conv_down_3 = double_conv(128, 256)
        self.conv_down_4 = double_conv(256, 512)
        self.conv_down_5 = double_conv(512, 1024)

        #convolute the image up
        self.trans_up_1 = nn.ConvTranspose2d(in_channels = 1024, out_channels = 512, kernel_size = 2, stride = 2)
        self.conv_up_1 = double_conv(1024, 512)

        self.trans_up_2 = nn.ConvTranspose2d(in_channels = 512, out_channels = 256, kernel_size = 2, stride = 2)
        self.conv_up_2 = double_conv(512, 256)

        self.trans_up_3 = nn.ConvTranspose2d(in_channels = 256, out_channels = 128, kernel_size = 2, stride = 2)
        self.conv_up_3 = double_conv(256, 128)

        self.trans_up_4 = nn.ConvTranspose2d(in_channels = 128, out_channels = 64, kernel_size = 2, stride = 2)
        self.conv_up_4 = double_conv(128, 64)

        self.out = nn.Conv2d(in_channels = 64, out_channels = 2, kernel_size= 1)



    def forward(self, img):
        x1 = self.conv_down_1(img)
        print(x1.size())
        x2 = self.max_pool(x1)
        x3 = self.conv_down_2(x2)
        x4 = self.max_pool(x3)
        x5 = self.conv_down_3(x4)
        x6 = self.max_pool(x5)
        x7 = self.conv_down_4(x6)
        x8 = self.max_pool(x7)
        x9 = self.conv_down_5(x8)
        print(x9.size())


        x10 = self.trans_up_1(x9)
        crop1 = image_cropping(x7,x10)
        x11 = self.conv_up_1(torch.cat[x10,crop1]) 

        x12 = self.trans_up_2(x11)
        crop2 = image_cropping(x5,x12)
        x13 = self.conv_up_2(torch.cat[x12,crop2]) 

        x14 = self.trans_up_3(x13)
        crop3 = image_cropping(x3,x14)
        x15 = self.conv_up_3(torch.cat[x14,crop3]) 

        x16 = self.trans_up_4(x15)
        crop4 = image_cropping(x1,x16)
        x17 = self.conv_up_4(torch.cat[x16,crop4]) 

        output = self.out(x17)
        print(x10.size())
        print(x7.size())
        print(crop1.size())


        return output
  

  
net = U_Net()
net = net.cuda()
print(net)

In [ ]:

# dummy data
rand_img = torch.rand((1,3,522,522))
rand_img = rand_img.cuda()

# test the forward pass
my_nn = U_Net()
my_nn = my_nn.cuda()
result = my_nn(img = rand_img)
print (result)

In [ ]:
def image_cropping(img, desired_img):
  desired_dimensions = desired_img.size()[2]
  current_dimensions = img.size()[2]
  difference = (current_dimensions - desired_dimensions)//2
  start_slice = difference
  end_slice = current_dimensions-difference
  img = img[:, :, start_slice:end_slice, start_slice:end_slice]
  return img

In [ ]:
rand_img = torch.rand((1,3,138,138))
rand_img2 = torch.rand((1,3,104,104))
img = image_cropping(rand_img,rand_img2)
print(img.size())